In [1]:
import os
import numpy as np
import netCDF4 as nc
import datetime
import matplotlib
import matplotlib.pyplot as plt
from scipy import optimize
import math

In [2]:
# Global Constants
g = 9.81 # m s^-2
Td1  = 24*3600+50*60               #M1 tidal period in seconds
Tm2  = 12*3600+25*60                # M2 tidal period in seconds

# Define standards for plots
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 200
matplotlib.rcParams['lines.linewidth'] = 1.5

In [3]:
def maximumfinder(a):
    if isinstance(a, float):
        amax = a
    else:
        amax = a.max()
    return amax

def stab_test(dt, dx, H):
    hmax = maximumfinder(H)
    CFL = (dt/dx)*(g*hmax)**.5
    
    if CFL < 1:
        return True
    else:
        return False

def simulation(deltaT=30, deltaX=500, Lbasin=2e4, Lb=4e4, B0=1e3, H0=10, M2amp=1, discharge=0, Cd=2.5e-3, constB=True, x_r=28e3, B_r=2.5e3):
    '''
    Function does numerical simulation with given boundary conditions and returns characteristic values/arrays.
    Solving the shallow water equations in shallow basins and estuaries.
    Based on Friedrichs (2011), Chapter 3 in Contemporary Issues in Estuarine Physics.
    
    Input:
        deltaT: time step in s
        deltaX: space step in m
        Lbasin: length basin in m
        Lb: e-folding length scale of basin in 1/m
        B0: width of basin on seaward side in m
        H0: basin depth in m
        M2amp: amplitude of M2 wave on seaward side in m
        discharge: constant discharge of river at landward boundary
        Cd: drag coefficient
        constB: should basin debpth be regarded as constant (=True) or with exponential decay (=False)?
        
    Ouput:
        Dictionary with information of interest about the simulation. Discharge and velocity have dimensions of x
        while Z has dimension of x -1.
    '''
    # defining constants
    time = np.arange(0, 10*Tm2+deltaT, deltaT) # time in seconds
    Nt   = len(time)                     # number of timesteps

    # preparing simulation
    x = np.arange(0, Lbasin+deltaX, deltaX)    # x in meters
    Nx = len(x)                                # number of horizontal locations
    B = np.full(len(x), B0)  # basin width [m]
    if not constB:
        B[x <= x_r] = B0 * np.exp(-x[x <= x_r]/Lb)
        B[x > x_r] = B_r
    H = np.ones(Nx)*H0        # depth [m]
    Z = np.zeros((Nx-1,Nt)) # Z points shifted half a grid point to the right with respect to Q points. we start with Q point, therefore 1 Z point less than Q points.      
    Q = np.zeros((Nx,Nt))
    B_H = np.array(B*H)                                           #volume 
    A = np.matmul(B_H.reshape(len(B_H),1), np.ones((1,Nt)))      # A at Q points 
    P = B.reshape(len(B),1) * np.ones((1,Nt))                    # Wetted perimeter at Q points.

    # Initalize Inertia, Pressure Gradient and Friction for further analysis later on.
    Inertia = np.zeros((Nx,Nt)) 
    PG      = np.zeros((Nx,Nt))
    Fric    = np.zeros((Nx,Nt))

    # Boundary conditions
    Z[0,:] = M2amp * np.sin(2*np.pi*time/Tm2)    # prescribed water levels. This is M2 tide.
    Q[Nx-1,:] = -discharge                       # river discharge; most often river discharge is taken zero.

    hmax = maximumfinder(H)
    courant = np.sqrt(g*hmax*deltaT/deltaX)

    # For numerical part, follow thesis of Speer (1984). Staggered grid. 
    # Z points shifted half deltaX to the right of U points: 
    # Q1 Z1 Q2 Z2 ........ ZN QN+1

    # solve Bdz/dt + dQ/dx=0
    # solve dQ/dt + d/dx Q^2/A = -gA dZ/dx - Cd Q|Q|P/A*A

    # Z is water level, B=estuary width, Q is discharge, A is cross-sectional area, 
    # P is wetted perimeter (~channel width when H/B is small)

    # Numerical scheme from Speer (1984).

    for pt in range(Nt):
        for px in range(2, Nx):
            Z[px-1,pt] = Z[px-1,pt-1] - (deltaT/(0.5*(B[px-1]+B[px])))*(Q[px,pt-1]-Q[px-1,pt-1])/deltaX
            A[px-1,pt] = B[px-1] * (H[px] + 0.5*Z[px-1,pt] + 0.5*Z[px-2,pt])           # A at Q points
            P[px-1,pt] = B[px-1] + 2*H[px-1] + Z[px-1,pt] + Z[px-2,pt]                 # P at Q points

            Inertia[px-1,pt]=(Q[px-1,pt]-Q[px-1,pt-1])/deltaT
            PG[px-1,pt]=-9.81*A[px-1,pt]*(1/deltaX)*(Z[px-1,pt]-Z[px-2,pt])
            Fric[px-1,pt]=-Cd*abs(Q[px-1,pt-1])*Q[px-1,pt-1]*P[px-1,pt-1]/(A[px-1,pt-1]*A[px-1,pt-1])

            Q[px-1,pt]= (Q[px-1,pt-1]                                           # Inertia
                - 9.81*A[px-1,pt]*(deltaT/deltaX)*(Z[px-1,pt]-Z[px-2,pt])       # Pressure gradient
                - Cd*deltaT*abs(Q[px-1,pt-1])*Q[px-1,pt-1]*P[px-1,pt-1]/(A[px-1,pt-1]*A[px-1,pt-1]))    # Friction

        Q[0,pt]=Q[1,pt]+B[0]*deltaX*(Z[0,pt]-Z[0,pt-1])/deltaT

    U=Q/A         # Flow velocity in m/s
    
    # test stability
    water_height = Z + hmax # TODO: oversimplified, arrays Z+H should be added properly
    if not stab_test(deltaT, deltaX, water_height):
        print("Unstable system!")       
    
    # define dictionary that is about to be returned, can be expanded if additional parameters are required
    returndict = {
        'Position': x, 'Time': time, 'Discharge': Q, 'Water_level': Z, 'Velocity': U
    }
    return returndict

# fit tidal constituents
def harmfit(invariant, *args):
    # a0, Cn, and Dn are the estimated amplitudes and phases of the constituents
    # wn are the angular frequencies of the consistuents
    # t is the time vector
    # output = mean + sum_k=1^k=Nharm Cn*sin(wn(k)*t) + Dn*cos(wn(k)*t)

    if len(args) == 1:
        args = tuple(args[0])

    t = invariant["timesteps"]
    wn = invariant["wn"]

    in_args = np.array(args[1:])

    a0 = args[0]
    Cn, Dn = np.split(in_args, 2)
    
    ts = np.array(t)
    
    test = np.empty(len(t))
    test.fill(a0)

    for k in range(len(Cn)):
        test = test + Cn[k] * np.sin(wn[k] * ts) + Dn[k] * np.cos(wn[k] * ts)
    return test

def tidefit(wn_vals, time_arr, ssh):
    '''
    Fits frequences to sea surface height.
    Input:
        wn_vals: array of frequencies to be fitted
        time_arr: arr of time points of the data to be fitted to.
        ssh: array sea surface height to which should be fitted. Same size as time_arr.
    Returns:
        array fitted parameters in form of coeffin and their respective covariance matrix.
    '''
    # fit parameters preparation
    a0 = [1]
    Cn, Dn = np.full(len(wn_vals), 1), np.full(len(wn_vals), 1)
    coefin = np.concatenate((a0, Cn, Dn))
    indep = {"timesteps": time_arr, "wn": wn_vals}

    # fit
    popt, pcov = optimize.curve_fit(harmfit, indep, ssh, coefin)
    return popt, pcov


def fit_sim_1x(t, z, x_idx=-1):
    '''
    Fits frequencies (for now just M2, M4, M6) to the time evolution of sea surface elevation at one position.
    Input:
        t: time array of data to be fitted
        z: array sea surface height with dimension #positions*#time-points. Same shape as from func "simulation".
        x_idx: index of position to be fitted to. Defaults to last index.
    '''
    per_vals = np.array([Tm2, Tm2/2, Tm2/3]) # period array with M2 Period and higher Harmonics
    wn_vals = 2 * np.pi / per_vals # frequencies f = 1/T converted from period
    popt, pcov = tidefit(wn_vals, t, z[x_idx])
    perr = np.sqrt(np.diag(pcov))
    return popt, perr

def par2amp(par, upar):
    '''
    Takes parameters as returned from func tidefit and returns the amplitude (and uncertainty) of each constituent.
    '''
    a0 = np.array([par[0]])
    u_a0 = np.array([upar[0]])
    Cn, Dn = np.split(par[1:], 2)
    u_Cn, u_Dn = np.split(upar[1:], 2)
    amplitude = np.sqrt(Cn ** 2 + Dn ** 2)
    np.seterr(invalid='ignore') #hacky way to not show warnings anymore
    u_amplitude = 1 / amplitude * np.sqrt((Cn * u_Cn)**2 + (Dn * u_Dn)**2)
    np.seterr(invalid='warn') # turn on warnings again
    return np.concatenate((a0, amplitude)), np.concatenate((u_a0, u_amplitude))

def par2phase(par, upar):
    '''
    Takes parameters as returned from func tidefit and returns the phase (and uncertainty) of each constituent.
    TODO: function not used yet
    '''
    a0 = par[0]
    u_a0 = upar[0]
    Cn, Dn = np.split(par[1:], 2)
    u_Cn, u_Dn = np.split(upar[1:], 2)
    phases = np.arctan2(Cn, Dn)
    np.seterr(invalid='ignore') #hacky way to not show warnings anymore
    # derivative from tan(x/y) is propto sec**2 = arccos**2
    u_phases = np.sqrt((u_Cn * np.arccos(Cn / Dn)**2 / Dn)**2 + (u_Dn * Cn * np.arccos(Cn / Dn) / Dn ** 2))
    # old: np.sqrt((Dn*u_Cn/(Cn**2 + Dn**2))**2 + (Cn*u_Dn/(Cn**2+Dn**2))**2)
    np.seterr(invalid='warn') # turn on warnings again                  
    return phases, u_phases
    
def get_amp_phase(x, t, z):
    '''
    Finds fitted amplitude and phase for every position x in z vector.
    '''
    length = len(z[:, 0])
    amp_arr = np.zeros(length, dtype=object)
    uamp_arr = np.zeros(length, dtype=object)
    phase_arr = np.zeros(length, dtype=object)
    uphase_arr = np.zeros(length, dtype=object)
    for xidx in range(length):
        popt, perr = fit_sim_1x(t, z, x_idx=xidx)
        amp_arr[xidx], uamp_arr[xidx] = par2amp(popt, perr)
        phase_arr[xidx], uphase_arr[xidx] = par2phase(popt, perr)
    return amp_arr, uamp_arr, phase_arr, uphase_arr

def rearr_arr(x, ux, xidx=0):
    '''
    Basically transposes array of array and returns only array with index idx. Defaults to returning only M2 amplitude.
    '''
    newx = np.zeros(len(x))
    newux = np.zeros(len(ux))
    for idx, item in enumerate(x):
        newx[idx] = item[xidx]
    for idx, item in enumerate(ux):
        newux[idx] = item[xidx]
    return newx, newux

def get_full_fit(x, t, z):
    '''
    Returns amplitudes (and uncertainties) and phases (and uncertainties) of all positions x in z-array).
    Input:
        x: Positon array of simulation where amplitudes are available.
        t: Time array of simulation where amplitudes are available.
        z: Sea surface height of simulation. Dimension x*t.
    Returns:
        Dictionary with all relevant fitting information. For each position point in x, a fit was done.
        Therefore, all arrays in the dictionary have the same size as len(z[:, 0]) which is len(x) or len(x)-1.
        The first is true for discharges etc while the latter is true for SSH.
    '''
    amp, uamp, phase, uphase = get_amp_phase(x, t, z)
    a0, ua0 = rearr_arr(amp, uamp, xidx=0)
    m2amp, um2amp = rearr_arr(amp, uamp, xidx=1)
    m4amp, um4amp = rearr_arr(amp, uamp, xidx=2)
    m6amp, um6amp = rearr_arr(amp, uamp, xidx=3)
    m2phase, um2phase = rearr_arr(phase, uphase, xidx=0)
    m4phase, um4phase = rearr_arr(phase, uphase, xidx=1)
    m6phase, um6phase = rearr_arr(phase, uphase, xidx=2)
    returndict = {"Mean Amplitude": a0, "M2 Amplitude": m2amp, "M4 Amplitude": m4amp, "M6 Amplitude": m6amp,
                  "Mean Amplitude Uncertainty": ua0, "M2 Amplitude Uncertainty": um2amp, "M4 Amplitude Uncertainty": um4amp, "M6 Amplitude Uncertainty": um6amp,
                  "M2 Phase": m2phase, "M4 Phase": m4phase, "M6 Phase": m6phase,
                  "M2 Phase Uncertainty": um2phase, "M4 Phase Uncertainty": um4phase, "M6 Phase Uncertainty": um6phase}
    return returndict